In [1]:
import os
import json 

# Thay đổi thư mục hiện tại
os.chdir('/home/ad/RoutingComparasion/RoutingComparation/routingcomparison')

# # Kiểm tra lại thư mục hiện tại đã được thay đổi chưa
# current_directory = os.getcwd()
# print("Đang đứng ở:", current_directory)
def read_data(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

from routingapp.compare_algorithm.sec_morl_multipolicy.train import train_sdn_policy
from routingapp.compare_algorithm.sec_morl_multipolicy.module_function import Function
from routingapp.compare_algorithm.sec_morl_multipolicy.module_graph import Graph
from routingapp.compare_algorithm.sec_morl_multipolicy.function_dijkstra import dijkstra, routing_k


# DFS function to find paths from source to destination
def dfs(graph, start, goal):
    visited = set()  # List of visited nodes
    stack = [(start, [start])]  # Stack containing pairs (node, path from source to node)

    while stack:
        node, path = stack.pop()  # Get the last node from the stack and the path to it
        if node not in visited:
            visited.add(node)
            if node == goal:
                return path  # Return the path from source to destination
            neighbors = graph.adj_matrix[node]  # Access neighbors from the adjacency matrix of the graph
            for neighbor in reversed(neighbors):  # Traverse neighbors in reverse order to use stack
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor]))  # Add unvisited neighbors to stack with path to that node

    return None  # If no path from source to destination is found


# Main function 
def run_sec(data_path, result_store):
    # Read data from JSON file
    data = read_data(data_path)
    nodes = data['nodes']
    edge_servers = data['server']
    cloud_servers = data['server']
    clients = data['client']
    edges = data['edges']
    adj_matrix = [[] for _ in range(len(nodes))]
    for edge in edges:
        adj_matrix[edge[0]].append(edge[1])
        adj_matrix[edge[1]].append(edge[0])

    # Define the graph object
    graph = Graph(len(nodes), len(clients), len(edge_servers), len(cloud_servers), len(clients), clients, edge_servers, cloud_servers, adj_matrix)

    # Initialize the function object
    func = Function()

    # Iterate through each scenario in the data
    scenario_list = data['scenario']
    for scenario in scenario_list:
        sum_delay_topo = 0
        delay_update = scenario["delay"]
        for delay in delay_update:
            sum_delay_topo  = sum_delay_topo + delay[2]
        bandwidth_update = scenario["bandwidth"]
        loss_update = scenario["loss"]
        request = scenario["request"]

        # Update the graph with delay, loss, and bandwidth information
        graph.updateGraph(delay_update, loss_update, bandwidth_update)

        print(graph.adj_matrix)
        # Generate the promising paths using DFS 
        promising_paths = []
        for src, dst in request:
            path = dfs(graph, src, dst)
            if path:
                promising_paths.append(path)

        # Create a subgraph containing all nodes in the promising paths
        promising_nodes = set()
        for path in promising_paths:
            promising_nodes.update(path)
        promising_graph = graph.subgraph(list(promising_nodes))

        print(promising_graph.adj_matrix)
        for req in request: 
            # print(req)
            # result = dijkstra(promising_graph, req[0], req[1], 1)
            print(result)
            # DRL to decide the best path for multi-objective reward
            trained_models = train_sdn_policy(promising_graph, func, req)
            
        # Use the trained models to generate solutions
        solutions = func.generate_solutions(promising_graph, func, request)  
        
        # Select the best solution
        result = func.select_solution(solutions)

    # Write the results to a JSON file
    with open(result_store, 'w') as f:
        json.dump(result, f)



In [2]:
run_sec(r"/home/ad/RoutingComparasion/RoutingComparation/routingcomparison/routingapp/compare_algorithm/data/oxford/5_server.json",r'result\sec_solver\oxford_5_server_2.json')

[[3, 11, 6], [11, 7], [3, 11], [0, 2], [5, 6], [4, 8], [0, 4], [1, 9], [5, 9], [7, 8], [18, 19], [0, 1, 2, 18, 15], [17, 13], [12, 14], [13, 16, 15], [11, 14, 16], [14, 15, 17], [12, 16, 18, 19], [10, 11, 17, 19], [10, 17, 18]]
[[3, 11, 6], [11, 7], [3, 11], [0, 2], [5, 6], [4, 8], [0, 4], [1, 9], [5, 9], [7, 8], [0, 1, 2, 18, 15], [17, 13], [12, 14], [13, 16, 15], [11, 14, 16], [14, 15, 17], [12, 16, 18, 19], [10, 11, 17, 19]]
[13, 7]


IndexError: index 19 is out of bounds for axis 0 with size 19